# import necessary packages

In [5]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import os
import numpy as np
from pandas import DataFrame


# features of each protein is located in 3 different sources that must be joined together


In [9]:
physical_features_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\Ligand+Receptor")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\Ligand+Receptor")
physical_features=[]
physicalname=[]
for file in range(0,len(physical_features_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\Ligand+Receptor")
    physicalname.append(physical_features_directory[file])
    dfpsaia=pd.read_csv(physical_features_directory[file])
   # print(dfpsaia)
    physical_features.append(dfpsaia)
print(len(physical_features))


162


In [11]:
chemical_features_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\chemical features")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\chemical features")
chemicalname=[]
chemical_features=[]
for file in range(0,len(chemical_features_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\chemical features")
    chemicalname.append(chemical_features_directory[file])
    chemical=pd.read_csv(chemical_features_directory[file])
    chemical=chemical.drop(['Unnamed: 0'],axis=1)
    #print(chemical)
    chemical_features.append(chemical)
print(len(chemical_features))

162


In [13]:
pssm_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\PSSM")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\PSSM")
pssm=[]
for file in range(0,len(pssm_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\PSSM")
    pssmfile=pd.read_csv(chemical_features_directory[file])
    pssmfile=pssmfile.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 1', 'Unnamed: 42','Unnamed: 43'],axis=1)
    
    print(pssmfile)
    pssm.append(pssmfile)
print(len(pssm))

    A  R  N  D  C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  W.1  \
0   0 -1  0 -1 -1 -1 -1 -2 -2 -1  ...    0    0    0    0    0    0    0    0   
1  -1  0  0  2 -4  2  5 -2  0 -3  ...    0    0    0    0    0    0    0    0   
2  -2 -3 -3 -4 -3 -3 -3 -3 -1  0  ...    0    0    0  100    0    0    0    0   
3   0 -2  0  3 -3 -1  0  5 -2 -4  ...    0    0    0    0    0    0    0    0   
4   0 -1  4  0 -1  0  0  0  0 -3  ...    0    0    0    0    0   68    0    0   
.. .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...  ...   
65  0 -2 -2 -2 -2 -2 -2 -3 -3  2  ...    0    0    0    0   12    0   44    0   
66 -2 -3 -3 -3 -4 -2 -2 -3 -3 -4  ...    0    0    0    0  100    0    0    0   
67  1  3 -1 -2 -2  1 -1  0  3 -2  ...    0    3    0    0    0    5   17    0   
68  0 -3 -4 -4 -2 -3 -3 -4 -4  4  ...    0    0    0    0    0    0    0    0   
69  0 -3 -1 -2 -3 -3 -3  6 -3 -5  ...    0    0    0    0    0    0    0    0   

    Y.1  V.1  
0     0    0

     A  R  N  D  C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0   -4 -6 -6 -6 -4 -5 -6 -6 -6  6  ...    2    0    0    0    0    0    0   
1   -3 -5 -6 -6 -4 -5 -5 -6 -6  5  ...    4    0    1    3    0    0    2   
2   -3 -5  1 -1 -6 -3 -2  7 -2 -7  ...    0    1    0    0    0    0    0   
3   -2 -6 -4 -5 -6 -5 -5  7 -5 -7  ...    0    0    0    0    0    0    0   
4   -2 -1 -2  4 -4  0  1 -2  2 -3  ...    1    1    0   14    0    5    4   
..  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
218 -2 -4 -3 -4 -3 -4 -4 -5 -2  5  ...    0    1    0    0    0    0   42   
219 -2 -5 -5 -6 -4 -4 -5 -6 -5  6  ...    7    0   26    0    0    0    1   
220  5  0 -1 -3 -3  1 -1 -2 -3 -4  ...    0    9    1    0    0    6    2   
221  2 -1  1 -1 -3 -2  0  4 -3 -4  ...    0    6    0    0    0   14    5   
222 -4 -3  8  0 -5 -3 -3 -3  2 -6  ...    0    0    0    0    0    3    0   

     W.1  Y.1  V.1  
0      0    0   38  
1      0    0   50  
2      0    

     A  R  N  D   C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0   -3 -3  2  7  -4 -1  1 -2 -2 -4  ...    0    0    0    0    0    0    0   
1   -2 -4 -4 -4  -2 -4 -4 -5 -4  5  ...    1    0    0    0    0    0    0   
2   -2 -1 -2 -3  -3  5 -1 -4 -2  0  ...    3    4    0    0    0    0    4   
3   -2 -2 -3 -4  -3  4 -2 -4 -3  0  ...   37    0   37    0    0    0    0   
4   -2 -3 -2 -3  -2 -3 -3 -4 -4 -1  ...   14    0    0    0    0    0   69   
..  .. .. .. ..  .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
429  0 -2  2 -1  -2 -1  1 -2 -2  2  ...    0    0    0    0    0   13    7   
430 -2  4 -1  3  -4 -1 -1 -2 -2 -4  ...    0    9    0    0   26   13    0   
431 -2  5  1 -2  -4  0 -1  0 -1 -3  ...    0   22    0    0    0    0   15   
432  0 -1  1  2   0  0  3 -2 -1 -3  ...    0    0    0    0    0   52    0   
433 -1 -4 -3 -4  10 -4 -4 -3 -4 -2  ...    0    0    0    0    0    0    0   

     W.1  Y.1  V.1  
0      0    0    0  
1      0    0   42  


     A  R  N  D  C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0   -2 -3 -4 -4 -2 -2 -3 -4 -3  2  ...   44    0   45    0    0    0    0   
1   -2  4 -1 -2 -3  2 -1 -3 -1 -3  ...    0   12    0    3    2    0   31   
2   -1 -1  0  2 -4  2  5 -3 -1 -3  ...    2    6    0    0    0    7   18   
3   -3  0 -3 -4 -4 -3 -4 -5  3 -2  ...    5    0    0   33    0    0    0   
4   -3  3 -2 -3 -5  0 -1 -4  0 -5  ...    0   85    0    2    0    0    0   
..  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
161  0  0 -1  2  0  3  4 -3 -2 -1  ...    2    4    0    0    0    2    1   
162 -2 -4 -5 -5 -3 -4 -5 -5 -5  6  ...    6    0    2    0    0    0    0   
163 -1  5  2 -2 -4  1  1 -3 -2 -1  ...    1   32    2    0    0    0    0   
164 -2  5 -1 -2 -5  3  2 -3  0 -4  ...    0   39    0    0    0    0    0   
165 -2 -2 -2 -3 -3 -2 -2 -3  4 -2  ...    0    0    0   13    0    0    0   

     W.1  Y.1  V.1  
0      0    0    0  
1      0    6    0  
2      0    

     A  R  N  D  C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0   -1  1  0  0 -3  5  3 -2  0 -3  ...    0    0    0    0    0    0    0   
1   -2 -2 -2 -3 -3 -2 -2 -3  2 -1  ...    0    0    0    0    0    0    0   
2   -1 -1 -2  0 -2  0  3 -3 -1  1  ...    0    0    0    0    0    0    0   
3   -2 -1  6  1 -3  0  0  0  1 -3  ...    0    0    0    0    0    0    0   
4    0  1  0 -1 -2  3  1 -2 -1 -2  ...    0   27    0    0    0    0   46   
..  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
170 -2 -2 -2 -3 -3 -2 -2 -3  2 -1  ...    0    0    0    0    0    0    0   
171 -1 -2 -3 -4 -1 -2 -3 -4 -3  1  ...    0    0    0    0    0    0    0   
172 -1  0  0  1 -4  2  5 -2  0 -3  ...    0    0    0    0    0    0    0   
173  0 -2 -3 -3 -1 -2 -2 -3 -3  2  ...    0    0    0    0    0    0    0   
174 -1 -2 -2 -1 -3 -1 -1 -2 -2 -3  ...    0    0    0    0    0    0    0   

     W.1  Y.1  V.1  
0      0    0    0  
1      0  100    0  
2      0    

[207 rows x 40 columns]
     A  R  N  D   C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0   -2  3 -1 -2  -5  0  0 -3 -2 -4  ...    0   87    0    0    0    0    0   
1   -1  0 -2 -1  -2  2 -2 -4 -3  3  ...    1    2    0    0    0    0   16   
2   -3 -3 -4 -5  -3 -3 -4 -4 -1  0  ...   22    0    8   39    0    0    0   
3   -1  0  0  1  -3  1  2  2 -1 -3  ...    0   10    0    0    2    6   33   
4   -3  6 -1 -3  -5  1 -1 -3  0 -3  ...    0   32    0    0    0    0    0   
..  .. .. .. ..  .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
124  1  3  0  3  -3  1  1 -2 -1 -3  ...    1   17    0    0    0    0    6   
125 -1 -3  1  0  -3 -2 -1  6 -2 -4  ...    1    0    0    0    0    4    0   
126 -2 -5 -5 -5  11 -5 -6 -4 -5 -3  ...    0    0    0    0    0    0    0   
127 -1  2  1  2  -4  0  1  2  2 -4  ...    0   24    0    0    0    3    3   
128 -2 -3 -4 -4  -2 -3 -4 -4 -4  1  ...   64    0    5    0    0    0    0   

     W.1  Y.1  V.1  
0      0    0    0

     A  R  N  D  C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0    0 -1  0 -1 -1 -1 -1 -2 -2 -1  ...    0    0    0    0    0    0  100   
1    2 -1  0 -1 -1  0 -1  0 -1 -2  ...    0    0    0    0    0   52   21   
2    2  0  0 -1 -1  0  0 -1  3 -2  ...    0   24    0    0    0   35    0   
3   -1 -3 -2 -2  9 -1  2 -3 -2 -2  ...    0    0    0    0    0    0    0   
4   -2  6 -1 -2 -4  1  0 -3 -1 -4  ...    0    0    0    0    0    0    0   
..  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
169 -1 -1  2 -1 -3  2  1 -2  6 -3  ...    0    0    0    0    7    0   15   
170  0 -3 -3 -3 -1 -2 -3 -3 -3  3  ...   25    0   10    0    0    0    6   
171  1 -1  3  1 -3  1  4 -1  0 -3  ...    0    0    0    0    0   17    0   
172 -1  3 -1  0 -4  1  4 -2  0 -3  ...    0   11    0    0    0    0    0   
173 -1  5 -1 -2 -4  1  0 -2 -1 -3  ...    0   29    0    0    0    0    0   

     W.1  Y.1  V.1  
0      0    0    0  
1      0    0    0  
2      0    

     A  R  N  D   C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0   -1 -5 -4 -5  10 -4 -5 -4 -4 -2  ...    0    0    0    0    0    0    0   
1    0 -1  0 -1  -2 -1 -1 -1 -2 -2  ...    0   12    0    0    0   55   34   
2   -1 -5 -4 -5  10 -4 -5 -4 -4 -2  ...    0    0    0    0    0    0    0   
3    2  1 -2 -2  -1 -1 -2 -2 -2  0  ...    0    0   16   11    0   15    0   
4   -2 -3 -3 -2  -4 -1  1 -3 -3 -3  ...    3    0    0    0   86    0    0   
..  .. .. .. ..  .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
121  2 -2 -2 -3  -1  0 -2 -2 -2  0  ...   34    0   17    0    0    0    0   
122  0 -2  3 -1  -3 -2 -2  6 -2 -4  ...    0    0    0    0    0    0    0   
123 -1 -4 -3 -4  10 -4 -4 -3 -4 -2  ...    0    0    0    0    0    0    0   
124 -1 -1  0  2  -4  2  5  0  0 -4  ...    0    0    0    0    0    0    0   
125 -1  0  0  2  -4  2  5 -2  0 -3  ...    0    0    0    0    0    0    0   

     W.1  Y.1  V.1  
0      0    0    0  
1      0    0    0  


[550 rows x 40 columns]
    A  R  N  D   C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  \
0  -2 -4 -4 -4  -2 -4 -4 -5 -4  4  ...    0    0    0    0    0    0    0   
1  -4 -3  3  7  -6 -2  1 -3 -3 -5  ...    0    0    0    0    0    0    0   
2  -4 -7 -6 -7  11 -6 -7 -6 -6 -4  ...    0    0    0    0    0    0    0   
3  -1 -1  0  2  -3 -2 -1 -1  0 -4  ...    0   12    0    0    1   51    3   
4  -2  0  2  2  -4  1  3  1  1 -2  ...    0   11    0    0    2    0    0   
5  -3 -3 -4 -5   3 -3 -4 -5  2 -1  ...    1    5    0   10    0    0    0   
6  -2  1 -2  0   2  0  1 -3 -2 -3  ...    1   12    0    0   38    4    9   
7  -2  1 -2 -2  -4  1  1 -1 -2 -3  ...    3   42    1    3    6   11    5   
8  -2 -2  0 -1  -4 -2 -2 -2  2 -3  ...    4   16    2    0   44    5    5   
9   2 -3 -1 -1  -2 -1 -2  2 -3  1  ...    6    3    3    3   10    2    2   
10 -4 -6 -6 -6  12 -6 -6 -6 -6 -4  ...    0    0    0    0    0    0    0   
11 -1 -3 -1 -3  -4 -1 -3 -3  0 -3  ...    3    6    

    A  R  N  D  C  Q  E  G  H  I  ...  L.1  K.1  M.1  F.1  P.1  S.1  T.1  W.1  \
0  -1 -2 -3 -3 -1 -1 -2 -3 -2  2  ...    0    0   66    0    0    0    0    0   
1  -1 -3 -4 -4 -1 -3 -3 -4 -4  3  ...   14    0    0    0    0    0    0    0   
2  -2 -3 -4 -4 -2 -3 -4 -4 -4  1  ...   69    0    0    0    0    0    0    0   
3   0 -1  1  3 -3  0  3 -2 -1 -3  ...    0    0    0    0    6   16    8    0   
4   2  2 -1 -2 -2 -1 -1  0 -2 -1  ...    7    5    0    0    0   19    0    0   
.. .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...  ...   
75  0  0  1 -2 -3  1  0 -2 -2 -2  ...    0   31    0    0    4   16   24    0   
76 -1  4 -2 -2 -4  2  0 -3 -2 -3  ...    4   17    0    0   16   12    7    0   
77  0  0  3  1 -3  0  0 -2 -2 -3  ...    3   39    0    0    1   11   12    0   
78 -2 -4 -4 -3 -5 -3 -3 -4 -4 -5  ...    0    0    0    0  100    0    0    0   
79  0 -3 -3 -3 -1 -2 -3 -3 -3  3  ...    0    0    0    0    0    0    0    0   

    Y.1  V.1  
0     0   34

# merging protein features from each directory

In [15]:
def mergeallfeatures(physical_features,chemical_features,pssm):
    all_features=[]
    letterabbreviation= {'CYS': 'C', 'ASP': 'D', 'GLN': 'Q', 'ILE': 'I',
                     'ALA': 'A', 'TYR': 'Y', 'TRP': 'W', 'HIS': 'H',
                     'LEU': 'L', 'ARG': 'R', 'VAL': 'V', 'GLU': 'E',
                     'PHE': 'F', 'GLY': 'G', 'MET': 'M', 'ASN': 'N',
                     'PRO': 'P', 'SER': 'S', 'LYS': 'K', 'THR': 'T',
                     # extended set of amino acids:
                     'MSE': 'M', 'CSE': 'U', 'LNT': 'X', 'GLH': 'E',
                     'HID': 'H', 'HIE': 'H', 'HIP': 'H', 'HYP': 'P'
                    }
    for i in range(len(pssm)):
        f=pd.merge(physical_features[i],chemical_features[i],right_index=True,left_index=True)
        features=pd.merge(f,pssm[i],right_index=True,left_index=True)
        #removing unnecessay columns
        features=features.drop(['ch total ASA|', 'ch b-bone ASA|', 'ch s-chain ASA|','ch polar ASA|','ch n-polar ASA|'],axis=1)
        letters = list(features['res name|'])
        features=features.drop(['res name|'],axis=1)
        letters = [letterabbreviation[base] for base in letters]
        v=''.join(letters)
        v=pd.DataFrame(letters,columns=['res name|'])
        feat=pd.merge(v,features,right_index=True,left_index=True)
        all_features.append(feat)
        print(feat)
    return all_features

In [16]:
all_features=mergeallfeatures(physical_features,chemical_features,pssm)

   res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0          T         *        1    152.6300      36.3200       116.310   
1          E         *        2    163.3900      26.8300       136.560   
2          F         *        3    180.0900      47.5660       132.520   
3          G         *        4     81.6460      81.6460         0.000   
4          S         *        5     95.0750      22.1910        72.883   
..       ...       ...      ...         ...          ...           ...   
65         V         *       66     71.5080      10.9220        60.586   
66         P         *       67      3.3792       3.3792         0.000   
67         H         *       68     96.8930       1.8035        95.089   
68         V         *       69     11.0870       2.7972         8.290   
69         G         *       70     12.4570      12.4570         0.000   

    polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.1  \
0      42.0880      110.5400    

[455 rows x 87 columns]
    res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           K         A       10     231.360      66.1140      165.2500   
1           F         A       11      35.343       0.7062       34.6370   
2           E         A       12      76.326       3.5167       72.8090   
3           S         A       13      59.878       7.6083       52.2690   
4           K         A       14      12.890       0.0000       12.8900   
..        ...       ...      ...         ...          ...           ...   
409         S         B      217      51.026      25.7680       25.2580   
410         L         B      218      54.738       3.9911       50.7470   
411         L         B      219     104.620      16.8360       87.7880   
412         T         B      220      16.046       8.2264        7.8193   
413         P         B      221      99.950      47.3890       52.5610   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.

    res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           C         A        1      72.985      69.1570        3.8281   
1           S         A        2     111.710      32.9710       78.7370   
2           C         A        3      57.386      33.2130       24.1730   
3           S         A        4      88.077      19.3550       68.7220   
4           P         A        5      60.772      20.4330       40.3390   
..        ...       ...      ...         ...          ...           ...   
177         Y         A      178      90.010       3.4543       86.5560   
178         R         A      179     117.910       9.2494      108.6600   
179         G         A      180      29.632      29.6320        0.0000   
180         A         A      181     107.990      39.9070       68.0870   
181         A         A      182      98.496      98.4960        0.0000   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.1  \
0        59.969    

[432 rows x 87 columns]
    res name| chain id| res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           R         *      82    255.5600      49.3090      206.2500   
1           D         *      83    107.1600       8.4297       98.7300   
2           F         *      84     26.4610       8.5278       17.9330   
3           N         *      85     26.0040       6.2984       19.7060   
4           N         *      86     61.0980       7.1093       53.9890   
..        ...       ...     ...         ...          ...           ...   
383         I         *     464      6.8528       0.3613        6.4915   
384         E         *     465    100.6800      11.4880       89.1880   
385         Y         *     466    135.2300      15.3490      119.8800   
386         F         *     467     20.7400      18.2520        2.4879   
387         L         *     468     97.3140      50.8070       46.5070   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.1  \
0      

[470 rows x 87 columns]
   res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0          T         B        2     103.890      75.7000       28.1850   
1          N         B        3      91.601      14.4070       77.1940   
2          L         B        4      29.644       1.2904       28.3530   
3          S         B        5      27.691       3.0930       24.5980   
4          D         B        6      89.117       8.1471       80.9700   
..       ...       ...      ...         ...          ...           ...   
78         K         B       80     134.950       6.3446      128.6100   
79         I         B       81      62.688      27.7490       34.9400   
80         K         B       82     106.350       4.2300      102.1200   
81         M         B       83      74.135      17.6840       56.4510   
82         L         B       84      46.392      38.0270        8.3644   

    polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.1  \
0      7

    res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           T         E      320    140.4100      58.9820       81.4280   
1           N         E      321    109.7100      15.2760       94.4360   
2           L         E      322     95.5280      17.7700       77.7580   
3           C         E      323      5.8740       5.1215        0.7525   
4           P         E      324     36.7260       8.6639       28.0620   
..        ...       ...      ...         ...          ...           ...   
178         V         E      498      0.0308       0.0308        0.0000   
179         L         E      499      1.7042       0.0000        1.7042   
180         S         E      500      9.9336       0.0000        9.9336   
181         F         E      501     17.9020      12.1370        5.7649   
182         E         E      502     97.1280      50.7260       46.4020   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.1  \
0       81.3000    

[110 rows x 87 columns]
    res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           Q         A        1    120.7500      49.9040       70.8470   
1           M         A        2     96.6250      20.6160       76.0090   
2           N         A        3     81.2550      15.3900       65.8650   
3           Y         A        4     11.6420       4.5319        7.1101   
4           E         A        5     69.9720       0.7907       69.1810   
..        ...       ...      ...         ...          ...           ...   
231         I         A      237      3.7306       0.0000        3.7306   
232         P         A      238      0.4065       0.4065        0.0000   
233         L         A      239     21.4340       3.9486       17.4860   
234         I         A      240     13.0400       8.7699        4.2706   
235         E         A      241     98.2020      25.0720       73.1310   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.

[427 rows x 87 columns]
    res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           V         A        3      84.943      72.3820        12.562   
1           R         A        4     200.420       7.3439       193.080   
2           S         A        5      49.083      23.3320        25.751   
3           L         A        6      71.567       1.2371        70.330   
4           N         A        7      31.593       5.3643        26.229   
..        ...       ...      ...         ...          ...           ...   
146         Q         A      149      50.230       3.8239        46.406   
147         F         A      150     136.460      26.9980       109.470   
148         V         A      151      50.435      17.3810        33.055   
149         S         A      152     124.010      51.3170        72.691   
150         S         A      153      71.824      69.2410         2.583   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.

[117 rows x 87 columns]
    res name| chain id|  res id|  total ASA|  b-bone ASA|  s-chain ASA|  \
0           G         A      429     78.1180      78.1180        0.0000   
1           E         A      430     75.3920      14.2250       61.1670   
2           L         A      431     21.7120       0.0000       21.7120   
3           R         A      432     20.8520       0.0000       20.8520   
4           V         A      433      0.7378       0.0000        0.7378   
..        ...       ...      ...         ...          ...           ...   
436         L         B      649     18.9800       8.0662       10.9140   
437         A         B      650     53.8520      15.6600       38.1920   
438         L         B      651      4.8328       1.3044        3.5284   
439         V         B      652     92.9580      18.0940       74.8650   
440         P         B      653    133.6500      81.8490       51.7990   

     polar ASA|  n-polar ASA|  total RASA|  b-bone RASA|  ...  L.1  K.1  M.

In [17]:
# simulating the interaction process between each cinsecutive two proteins
receptor_ligand=[]
for i in range(0,len(all_features),2):
    ligand=all_features[i]
    receptor=all_features[i+1]
    ligandlenght=len(ligand)
    r=receptor.loc[receptor.index.repeat(ligandlenght)].reset_index(drop=True)
    receptorlenght=len(receptor)
    l= pd.concat([ligand]*receptorlenght, ignore_index=True)
    df=pd.merge(r,l,left_index=True,right_index=True)
    print(df)
    receptor_ligand.append(df)
    

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               C           B          1       100.810         60.470   
1               C           B          1       100.810         60.470   
2               C           B          1       100.810         60.470   
3               C           B          1       100.810         60.470   
4               C           B          1       100.810         60.470   
...           ...         ...        ...           ...            ...   
17145           N           B        245        68.493         26.952   
17146           N           B        245        68.493         26.952   
17147           N           B        245        68.493         26.952   
17148           N           B        245        68.493         26.952   
17149           N           B        245        68.493         26.952   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0              40.342        51.226          49.586  

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               Q           A        566        214.62         72.950   
1               Q           A        566        214.62         72.950   
2               Q           A        566        214.62         72.950   
3               Q           A        566        214.62         72.950   
4               Q           A        566        214.62         72.950   
...           ...         ...        ...           ...            ...   
76853           P           A       1045        141.05         65.726   
76854           P           A       1045        141.05         65.726   
76855           P           A       1045        141.05         65.726   
76856           P           A       1045        141.05         65.726   
76857           P           A       1045        141.05         65.726   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0              141.67       155.240          59.378  

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               K           B          1        236.12         85.847   
1               K           B          1        236.12         85.847   
2               K           B          1        236.12         85.847   
3               K           B          1        236.12         85.847   
4               K           B          1        236.12         85.847   
...           ...         ...        ...           ...            ...   
56539           V           B        124        106.71         33.530   
56540           V           B        124        106.71         33.530   
56541           V           B        124        106.71         33.530   
56542           V           B        124        106.71         33.530   
56543           V           B        124        106.71         33.530   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0             150.270        99.378         136.740  

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               G           A          1        131.43        131.430   
1               G           A          1        131.43        131.430   
2               G           A          1        131.43        131.430   
3               G           A          1        131.43        131.430   
4               G           A          1        131.43        131.430   
...           ...         ...        ...           ...            ...   
34123           L           A        316        135.59         46.475   
34124           L           A        316        135.59         46.475   
34125           L           A        316        135.59         46.475   
34126           L           A        316        135.59         46.475   
34127           L           A        316        135.59         46.475   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0               0.000        81.572          49.859  

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               N           B          1        110.25         47.981   
1               N           B          1        110.25         47.981   
2               N           B          1        110.25         47.981   
3               N           B          1        110.25         47.981   
4               N           B          1        110.25         47.981   
...           ...         ...        ...           ...            ...   
69001           C           C        220        103.03         51.925   
69002           C           C        220        103.03         51.925   
69003           C           C        220        103.03         51.925   
69004           C           C        220        103.03         51.925   
69005           C           C        220        103.03         51.925   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0              62.272        79.178          31.075  

...

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               M           A          1        162.10         18.088   
1               M           A          1        162.10         18.088   
2               M           A          1        162.10         18.088   
3               M           A          1        162.10         18.088   
4               M           A          1        162.10         18.088   
...           ...         ...        ...           ...            ...   
12477           S           A        158        133.65         95.003   
12478           S           A        158        133.65         95.003   
12479           S           A        158        133.65         95.003   
12480           S           A        158        133.65         95.003   
12481           S           A        158        133.65         95.003   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0             144.010        16.429         145.670  

       res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0                E           H          1       142.550         74.928   
1                E           H          1       142.550         74.928   
2                E           H          1       142.550         74.928   
3                E           H          1       142.550         74.928   
4                E           H          1       142.550         74.928   
...            ...         ...        ...           ...            ...   
192460           G           L        212        27.768         27.768   
192461           G           L        212        27.768         27.768   
192462           G           L        212        27.768         27.768   
192463           G           L        212        27.768         27.768   
192464           G           L        212        27.768         27.768   

        s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0               67.619       59.3470    

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               M           A          1        101.03         31.476   
1               M           A          1        101.03         31.476   
2               M           A          1        101.03         31.476   
3               M           A          1        101.03         31.476   
4               M           A          1        101.03         31.476   
...           ...         ...        ...           ...            ...   
54091           P           A        294        143.49         77.532   
54092           P           A        294        143.49         77.532   
54093           P           A        294        143.49         77.532   
54094           P           A        294        143.49         77.532   
54095           P           A        294        143.49         77.532   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0              69.549        29.359          71.667  

      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               Q           H          1        167.91         50.289   
1               Q           H          1        167.91         50.289   
2               Q           H          1        167.91         50.289   
3               Q           H          1        167.91         50.289   
4               Q           H          1        167.91         50.289   
...           ...         ...        ...           ...            ...   
19275           E           L        112         92.42         62.926   
19276           E           L        112         92.42         62.926   
19277           E           L        112         92.42         62.926   
19278           E           L        112         92.42         62.926   
19279           E           L        112         92.42         62.926   

       s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0             117.620       107.000          60.903  

       res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0                E           H          1       170.060         43.972   
1                E           H          1       170.060         43.972   
2                E           H          1       170.060         43.972   
3                E           H          1       170.060         43.972   
4                E           H          1       170.060         43.972   
...            ...         ...        ...           ...            ...   
185720           C           L        218        41.224         27.925   
185721           C           L        218        41.224         27.925   
185722           C           L        218        41.224         27.925   
185723           C           L        218        41.224         27.925   
185724           C           L        218        41.224         27.925   

        s-chain ASA|_x  polar ASA|_x  n-polar ASA|_x  total RASA|_x  \
0              126.090        98.059    

In [30]:
print(len(receptor_ligand))

81


In [18]:
import pandas as pd
import os
import numpy as np
label_directory = os.listdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\negative and positive samples")
os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\negative and positive samples")
label=[]
labelname=[]
for file in range(0,len(label_directory)) :
    os.chdir(r"C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\negative and positive samples")
    samples=pd.read_csv(label_directory[file])
    label.append(samples)
    labelname.append(label_directory[file])


C:\Users\ahmed hatem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# each protien has a structure before interaction (where features are extracted) that is different that it's structure after interaction (where label or target is extracted using global alignment we were able to find the common strucutre between the two

In [6]:
def GET_SCORE(n1, n2, penalty = -1, reward = 1):
     
    if n1 == n2:
        return reward
    else:
        return penalty
 
 
def global_alignment(X, Y, penalty = -1, reward = 1):
     
    # initialize score matrix
    score_matrix = np.ndarray((len(X) + 1, len(Y) + 1))
      
    for i in range(len(X) + 1):
        score_matrix[i, 0] = penalty * i
     
    for j in range(len(Y) + 1):
        score_matrix[0, j] = penalty * j
         
     
    # define each cell in the matrix by as the max score possible in that stage
    for i in range(1, len(X) + 1):
        for j in range(1, len(Y) + 1):
            match = score_matrix[i - 1, j - 1] + GET_SCORE(X[i - 1], Y[j - 1], penalty, reward)
            delete = score_matrix[i -1, j] + penalty
            insert = score_matrix[i, j - 1] + penalty
             
            score_matrix[i, j] = max([match, delete, insert])
             
     
    i = len(X)
    j = len(Y)
     
    global_alignment.align_X = ""
    global_alignment.align_Y = ""
     
    while i > 0 or j > 0:
         
        current_score = score_matrix[i, j]
        left_score = score_matrix[i - 1, j]
         
         
        if i > 0 and j > 0 and X[i - 1] == Y[j - 1]:
            global_alignment.align_X = X[i - 1] + global_alignment.align_X
            global_alignment.align_Y = Y[j - 1] + global_alignment.align_Y
            i = i - 1
            j = j - 1
         
        elif i > 0 and current_score == left_score + penalty:
            global_alignment.align_X = X[i - 1] + global_alignment.align_X
            global_alignment.align_Y = '*' + global_alignment.align_Y
            i = i - 1
             
        else:
            global_alignment.align_X = '*' + global_alignment.align_X
            global_alignment.align_Y = Y[j - 1] + global_alignment.align_Y
            j = j - 1
 
 
    return global_alignment.align_X, global_alignment.align_Y

In [22]:
def disorder_removing(label,receptor_ligand):
    disorderremoved=[]
    for i in range(0,len(label)):
        print(labelname[i])

        feature=receptor_ligand[i]
        sample=label[i]
        df1=feature
        z=df1['res id|_y']
        f=1
        for i in range(len(z)-1):
            if z[i]<z[i+1]:
                f+=1
        
            else:
                break
        s=df1['res name|_y'][:f]
        a = list(s)
        x = ''.join(str(e) for e in a)
    #print(x)
        df=sample
        d=df['structResIdL']
        q=1
        for i in range(len(d)-1):
            if d[i]<d[i+1]:
                q+=1
        
            else:
                break
        m=df['resNameL'][:q]
        a = list(m)
        y = ''.join(str(e) for e in a)
    #print(y)
        global_alignment(x, y)
        ax=global_alignment.align_X
        ay=global_alignment.align_Y
        print(ax)
        print(ay)
        colum=" ".join(ax)
        colum=colum.split()
        colum1=" ".join(ay)
        colum1=colum1.split()
        dfax = DataFrame (colum,columns=['alignmx'])
        d1=df1['res id|_x']
        c=1
        for d in range(len(d1)-1):
    
            if d1[d]!=d1[d+1]:
                c+=1
        df_repeated = pd.concat([dfax]*c, ignore_index=True)
    #print(df_repeated)
        v=df_repeated.index[df_repeated['alignmx'] == '*'].tolist()
        h=0
        dfa=np.array(df1)
        cols=df1.columns

        l=[]
        for i in v:
            if i<=len(dfa):
            
                dfa=np.insert(dfa,i,"nan",axis = 0)
  
        if v:
            
            lu=pd.DataFrame(dfa,columns=cols)
        else :
            lu=pd.DataFrame(df1,columns=cols)
        lu['alignmx']=df_repeated
        #lu=pd.DataFrame(df_repeated,columns=cols)
        
        dfay1 = DataFrame(colum1,columns=['alignmy'])
        df_repeated = pd.concat([dfay1]*c, ignore_index=True)
        lu["aligny"]=df_repeated
        print(lu)
        lu['Match'] = np.where(lu['aligny'] == "*", 'True', 'False') 
        index_names = lu[lu['Match'] =="True"].index
        lu.drop(index_names, inplace = True)
        lu['Match'] = np.where(lu == "nan", 'True', 'False') 
        index_names = lu[lu['Match'] =="True"].index
        lu.drop(index_names, inplace = True)

        df_repeated = pd.concat([dfay1]*c, ignore_index=True)
    #print(df_repeated)
        v=df_repeated.index[df_repeated['alignmy'] == '*'].tolist()
        w=0
        dfa=np.array(df)
        for i in v:
            if i<=len(dfa):
                dfa=np.insert(dfa,i,"nan",axis = 0)
    #x=np.insert(dfa,l,'nan',axis=0)

        cols=df.columns
        if v:
        
            lb=pd.DataFrame(dfa,columns=cols)
        else:
            lb=df
        lb['alignmy']=df_repeated
        lb=lb.dropna(axis=0)
        df_repeated = pd.concat([dfax]*c, ignore_index=True)
        lb['alignmx']=df_repeated
        lb['Match'] = np.where(lb['alignmx'] == "*", 'True', 'False') 
        index_names = lb[lb['Match'] =="True"].index
        lb.drop(index_names, inplace = True)
        lb=lb.dropna(axis=0)
        index_names = lb[lb['Match'] =="True"].index
        lb.drop(index_names, inplace = True)
        lb['Match'] = np.where(lb == 'nan', 'True', 'False')
        index_names = lb[lb['Match'] =="True"].index
        lb.drop(index_names, inplace = True)
        df3 = pd.merge(lu, lb, left_index=True, right_index=True)
    #print(df3)
        df=lu
        df1=lb
        m=df['res name|_x']
        a = list(m)
        x = ''.join(str(e) for e in a)

        g=df1['resNameR']
        a = list(g)
        y = ''.join(str(e) for e in a)
        df1.reset_index(inplace = True)

        d1=df1['structResIdR']
        li=[]
        for i in range(len(d1)-1):
    
             if d1[i]!=d1[i+1]:
                li.append(a[i])
        li.append(a[-1])
        ah= ''.join(str(e) for e in li)
        df.reset_index(inplace = True)
        d1=df['res id|_x']
        li=[]
        for i in range(len(d1)-1):
    
             if d1[i]!=d1[i+1]:
                li.append(x[i])
        li.append(x[-1])
        x = ''.join(str(e) for e in li)
        global_alignment(x,ah)
        ax=global_alignment.align_X
        ay=global_alignment.align_Y
        print(ax)
        print(ay)
        colum=" ".join(ax)
        colum=colum.split()
        colum1=" ".join(ay)
        colum1=colum1.split()
        dfax = DataFrame (colum,columns=['alignmx'])
        alignmx=dfax.loc[dfax.index.repeat(f)].reset_index(drop=True)
        v=alignmx.index[alignmx['alignmx'] == '*'].tolist()
        h=0
        dfa=np.array(df1)
        l=[]

    #print(v)
        for i in v:
            if i<=len(dfa):
                dfa=np.insert(dfa,i,"nan",axis = 0)
    #x=np.insert(dfa,l,'nan',axis=0)
      
   # x=np.insert(dfa,v,'nan',axis=0)
        cols=df1.columns
        if v:
            ru=pd.DataFrame(dfa,columns=cols)
        else:
            ru=df1
        dfay1 = DataFrame(colum1,columns=['alignmy'])
        alignmy=dfay1.loc[dfay1.index.repeat(f)].reset_index(drop=True)
        print(alignmy)
        v=alignmx.index[alignmx['alignmx'] == '*'].tolist()
        h=0
        dfa=np.array(df)

        for i in v:
            if i<=len(dfa):
            
                dfa=np.insert(dfa,i,"nan",axis = 0)
        
    
    #x=np.insert(dfa,v,'nan',axis=0)
        cols=df.columns
        if v:
            ru=pd.DataFrame(dfa,columns=cols)
        else :
            ru=df
        ru['alignmx']=alignmx
        ru["alignmy"]=alignmy
        ru['Match'] = np.where(ru['alignmy'] == "*", 'True', 'False') 
        index_names = ru[ru['Match'] =="True"].index
        ru.drop(index_names, inplace = True)
        ru['Match'] = np.where(ru == "nan", 'True', 'False') 
        index_names = ru[ru['Match'] =="True"].index
        ru.drop(index_names, inplace = True)
        v=alignmy.index[alignmy['alignmy'] == '*'].tolist()
        h=0
        dfa=np.array(df1)

        for i in v:
            if i<=len(dfa):
            
                dfa=np.insert(dfa,i,"nan",axis = 0)
            
    #x=np.insert(dfa,l,'nan',axis=0)

    #x=np.insert(dfa,v,'nan',axis=0)
        cols=df1.columns
        if v:
            rb=pd.DataFrame(dfa,columns=cols)
        else:
            rb=df1
        rb['alignmy']=alignmy
        rb['alignmx']=alignmx
        rb['Match'] = np.where(rb['alignmx'] == "*", 'True', 'False') 
        index_names = rb[rb['Match'] =="True"].index
        rb.drop(index_names, inplace = True)
        rb['Match'] = np.where(rb== 'nan', 'True', 'False') 
        index_names = rb[rb['Match'] =="True"].index
        rb.drop(index_names, inplace = True)
        ru.reset_index(inplace=True)
        rb.reset_index(inplace=True)
        df3 = pd.merge(ru, rb, left_index=True, right_index=True)
        df3.dropna(axis=0,inplace=True)

        disorderremoved.append(df3)
    return disorderremoved

In [23]:
removed=disorder_removing(label,receptor_ligand)

1ACB.csv
TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNHVPHVG
*******KSFPEVVGKTVDQAREYFTLHYPQYDVYFLPEGSPVTLDLRYNRVRVFYNPGTNVVNHVPHVG
      res name|_x chain id|_x  res id|_x  total ASA|_x  b-bone ASA|_x  \
0               C           B          1       100.810         60.470   
1               C           B          1       100.810         60.470   
2               C           B          1       100.810         60.470   
3               C           B          1       100.810         60.470   
4               C           B          1       100.810         60.470   
...           ...         ...        ...           ...            ...   
17145           N           B        245        68.493         26.952   
17146           N           B        245        68.493         26.952   
17147           N           B        245        68.493         26.952   
17148           N           B        245        68.493         26.952   
17149           N           B        245      

<ipython-input-22-9999d85e5f02>:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lb['alignmx']=df_repeated
<ipython-input-22-9999d85e5f02>:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lb['Match'] = np.where(lb['alignmx'] == "*", 'True', 'False')
C:\Users\ahmed hatem\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDGETRKVKAHSQTHRVDLGTLRGYYNQSEAGSHTVQRMYGCDVGSDWRFLRGYHQYAYDGKDYIALKEDLRSWTAADMAAQTTKHKWEAAHVAEQLRAYLEGTCVEWLRRYLENGKETLQRTDAPKTHMTHHAVSDHEATLRCWALSFYPAEITLTWQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGQEQRYTCHVQHEGLPKPLTLRWEMIQRTPKIQVYSRHPAENGKSNFLNCYVSGFHPSDIEVDLLKNGERIEKVEHSDLSFSKDWSFYLLYYTEFTPTEKDEYACRVNHVTLSQPKIVKWDRDM
GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDGETRKVKAHSQTHRVDLGTLRGYYNQSEAGSHTVQRMYGCDVGSDWRFLRGYHQYAYDGKDYIALKEDLRSWTAADMAAQTTKHKWEAAHVAEQLRAYLEGTCVEWLRRYLENG**********K*************************E*TL**QR*************T****D****************************************************A****************P*************************K**************T**********H***************
      alignmy
0           G
1           G
2           G
3           G
4           G
...       ...
42745       *
42746       *
42747       *
42748       *
42749       *

[42750 rows x 1 columns]


KeyboardInterrupt: 

In [151]:
#os.chdir(r'C:\Users\ahmed hatem\Downloads\GP\disorder removed')
#for i in range(len(disorderremoved)):
    #os.chdir(r'C:\Users\ahmed hatem\Downloads\GP\disorder removed')

    #disorderremoved[i].to_csv(labelname[i])

In [198]:
readyforundersampling=[]
for i in range(0,len(temp)):
    
    protein=temp[i]
    
    protein.dropna(axis=0,inplace=True)

    
    protein=protein.drop(['index_x', 'level_0_x','level_0_y','res id|_x','res id|_y','chain id|_x','chain id|_y','alignmx_x', 'aligny', 'Match_x',
       'alignmy_x',  'chainIdL', 'structResIdL',
       'resNameL', 'chainIdR', 'structResIdR', 'resNameR','alignmx_y', 'Match_y','alignmy_y'],axis=1)
    print(protein)
    readyforundersampling.append(protein)
readyforundersampling

      res name|_x  total ASA|_x  b-bone ASA|_x  s-chain ASA|_x  polar ASA|_x  \
0               C       100.810         60.470          40.342        51.226   
1               C       100.810         60.470          40.342        51.226   
2               C       100.810         60.470          40.342        51.226   
3               C       100.810         60.470          40.342        51.226   
4               C       100.810         60.470          40.342        51.226   
...           ...           ...            ...             ...           ...   
15150           N        68.493         26.952          41.542        47.876   
15151           N        68.493         26.952          41.542        47.876   
15152           N        68.493         26.952          41.542        47.876   
15153           N        68.493         26.952          41.542        47.876   
15154           N        68.493         26.952          41.542        47.876   

       n-polar ASA|_x  total RASA|_x  b

      res name|_x  total ASA|_x  b-bone ASA|_x  s-chain ASA|_x  polar ASA|_x  \
0               G        52.934         52.934           0.000        32.562   
1               G        52.934         52.934           0.000        32.562   
2               G        52.934         52.934           0.000        32.562   
3               G        52.934         52.934           0.000        32.562   
4               G        52.934         52.934           0.000        32.562   
...           ...           ...            ...             ...           ...   
34877           E       102.750         53.870          48.876        60.251   
34878           E       102.750         53.870          48.876        60.251   
34879           E       102.750         53.870          48.876        60.251   
34880           E       102.750         53.870          48.876        60.251   
34881           E       102.750         53.870          48.876        60.251   

       n-polar ASA|_x  total RASA|_x  b

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               I       171.68        68.975          102.7       62.807   
1               I       171.68        68.975          102.7       62.807   
2               I       171.68        68.975          102.7       62.807   
3               I       171.68        68.975          102.7       62.807   
4               I       171.68        68.975          102.7       62.807   
...           ...          ...           ...            ...          ...   
12755           N       45.675        7.8077         37.868       38.768   
12756           N       45.675        7.8077         37.868       38.768   
12757           N       45.675        7.8077         37.868       38.768   
12758           N       45.675        7.8077         37.868       38.768   
12759           N       45.675        7.8077         37.868       38.768   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0             108.

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               A       151.38        94.461         56.916        73.03   
1               A       151.38        94.461         56.916        73.03   
2               A       151.38        94.461         56.916        73.03   
3               A       151.38        94.461         56.916        73.03   
4               A       151.38        94.461         56.916        73.03   
...           ...          ...           ...            ...          ...   
17176           A       30.198        1.6464         28.552            0   
17177           A       30.198        1.6464         28.552            0   
17178           A       30.198        1.6464         28.552            0   
17179           A       30.198        1.6464         28.552            0   
17180           A       30.198        1.6464         28.552            0   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0             78.3

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               P       53.718        26.184         27.535       26.112   
1               P       53.718        26.184         27.535       26.112   
2               P       53.718        26.184         27.535       26.112   
3               P       53.718        26.184         27.535       26.112   
4               P       53.718        26.184         27.535       26.112   
...           ...          ...           ...            ...          ...   
23011           S       55.901        15.323         40.578       45.664   
23012           S       55.901        15.323         40.578       45.664   
23013           S       55.901        15.323         40.578       45.664   
23014           S       55.901        15.323         40.578       45.664   
23015           S       55.901        15.323         40.578       45.664   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0             27.6

...

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               Q       210.01        63.402         146.61       134.69   
1               Q       210.01        63.402         146.61       134.69   
2               Q       210.01        63.402         146.61       134.69   
3               Q       210.01        63.402         146.61       134.69   
4               Q       210.01        63.402         146.61       134.69   
...           ...          ...           ...            ...          ...   
64363           G       82.094        82.094              0       3.4928   
64364           G       82.094        82.094              0       3.4928   
64365           G       82.094        82.094              0       3.4928   
64366           G       82.094        82.094              0       3.4928   
64367           G       82.094        82.094              0       3.4928   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0             75.3

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               Q       134.94         73.22         61.723       57.941   
1               Q       134.94         73.22         61.723       57.941   
2               Q       134.94         73.22         61.723       57.941   
3               Q       134.94         73.22         61.723       57.941   
4               Q       134.94         73.22         61.723       57.941   
...           ...          ...           ...            ...          ...   
30310           S       97.441         34.36          63.08       51.846   
30311           S       97.441         34.36          63.08       51.846   
30312           S       97.441         34.36          63.08       51.846   
30313           S       97.441         34.36          63.08       51.846   
30314           S       97.441         34.36          63.08       51.846   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0             77.0

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               E       187.42        61.403         126.02       125.28   
1               E       187.42        61.403         126.02       125.28   
2               E       187.42        61.403         126.02       125.28   
3               E       187.42        61.403         126.02       125.28   
4               E       187.42        61.403         126.02       125.28   
...           ...          ...           ...            ...          ...   
47229           S       11.759             0         11.759       9.4553   
47230           S       11.759             0         11.759       9.4553   
47231           S       11.759             0         11.759       9.4553   
47232           S       11.759             0         11.759       9.4553   
47233           S       11.759             0         11.759       9.4553   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0             62.1

      res name|_x total ASA|_x b-bone ASA|_x s-chain ASA|_x polar ASA|_x  \
0               Q       166.46         57.59         108.87       112.03   
1               Q       166.46         57.59         108.87       112.03   
2               Q       166.46         57.59         108.87       112.03   
3               Q       166.46         57.59         108.87       112.03   
4               Q       166.46         57.59         108.87       112.03   
...           ...          ...           ...            ...          ...   
54985           S        96.85        56.574         40.275        28.41   
54986           S        96.85        56.574         40.275        28.41   
54987           S        96.85        56.574         40.275        28.41   
54988           S        96.85        56.574         40.275        28.41   
54989           S        96.85        56.574         40.275        28.41   

      n-polar ASA|_x total RASA|_x b-bone RASA|_x s-chain RASA|_x  \
0              54.

[      res name|_x  total ASA|_x  b-bone ASA|_x  s-chain ASA|_x  polar ASA|_x  \
 0               C       100.810         60.470          40.342        51.226   
 1               C       100.810         60.470          40.342        51.226   
 2               C       100.810         60.470          40.342        51.226   
 3               C       100.810         60.470          40.342        51.226   
 4               C       100.810         60.470          40.342        51.226   
 ...           ...           ...            ...             ...           ...   
 15150           N        68.493         26.952          41.542        47.876   
 15151           N        68.493         26.952          41.542        47.876   
 15152           N        68.493         26.952          41.542        47.876   
 15153           N        68.493         26.952          41.542        47.876   
 15154           N        68.493         26.952          41.542        47.876   
 
        n-polar ASA|_x  to

In [199]:
len(readyforundersampling)

81

In [4]:
readyforundersampling=[]
di=os.listdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\disorder removed')
os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\disorder removed')
for i in range(len(di)):
    os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\disorder removed')

    df=pd.read_csv(di[i])
    df=df.drop(['index_x', 'level_0_x','level_0_y','res id|_x','res id|_y','chain id|_x','chain id|_y','alignmx_x', 'aligny', 'Match_x',
       'alignmy_x',  'chainIdL', 'structResIdL',
       'resNameL', 'chainIdR', 'structResIdR', 'resNameR','alignmx_y', 'Match_y','alignmy_y','Unnamed: 0'],axis=1)

    readyforundersampling.append(df)

C:\Users\ahmed hatem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,187) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ahmed hatem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ahmed hatem\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (187) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
readyforundersampling

[      res name|_x  total ASA|_x  b-bone ASA|_x  s-chain ASA|_x  polar ASA|_x  \
 0               C       100.810         60.470          40.342        51.226   
 1               C       100.810         60.470          40.342        51.226   
 2               C       100.810         60.470          40.342        51.226   
 3               C       100.810         60.470          40.342        51.226   
 4               C       100.810         60.470          40.342        51.226   
 ...           ...           ...            ...             ...           ...   
 15150           N        68.493         26.952          41.542        47.876   
 15151           N        68.493         26.952          41.542        47.876   
 15152           N        68.493         26.952          41.542        47.876   
 15153           N        68.493         26.952          41.542        47.876   
 15154           N        68.493         26.952          41.542        47.876   
 
        n-polar ASA|_x  to

In [6]:
def undersampling(readyforundersampling):
    undersampleddata=[]
    for a in range(len(readyforundersampling)):
        #print(readyforundersampling)
        protein=readyforundersampling[a]        
        undersample = RandomUnderSampler()
       # y_under=protein['prediction']
        x = protein.iloc[:, 0 :170]
        y = protein.iloc[:, [171]]
        print(y)
        z=y['prediction']
        for v in range(len(z)):
            if z[v]<0.6:
                z[v]=0
            else:
                z[v]=1
        z=z.astype('float')
        x_under, y_under = undersample.fit_resample(x, z)
    
        #encoding={'A':10000000000000000000,'L':0o01000000000000000000,'I':0o00100000000000000000,'V':0o00010000000000000000,'G':0o00001000000000000000,'K':0o00000100000000000000,'R':0o00000010000000000000,'D':0o00000001000000000000,'E':0o00000000100000000000,'H':0o00000000010000000000,'N':0o00000000001000000000,'Q':0o00000000000100000000,'S':0o00000000000010000000,'T':0o00000000000001000000,'C':0o00000000000000100000,'M':0o00000000000000010000,'Y':0o00000000000000001000,'W':0o00000000000000000100,'F':0o00000000000000000010,'P':0o00000000000000000001}

        #x_under['res name|_x'] = [encoding[base] for base in x_under['res name|_x']]
        #x_under['res name|_y'] = [encoding[base] for base in x_under['res name|_y']]
        #x_under=pd.DataFrame(x_under)
        #y_under=pd.DataFrame(y_under)
        undersampled=pd.merge(x_under,y_under,left_index=True,right_index=True)
        undersampl=shuffle(undersampled)
        #undersampl=undersampl.reset_index(inplace=True)

        #print(undersampl )

        undersampleddata.append(undersampl)
    return undersampleddata

In [7]:
undersampleddata=undersampling(readyforundersampling)

       prediction
0        0.001141
1        0.000016
2        0.000027
3        0.000023
4        0.000006
...           ...
15150    0.000129
15151    0.000265
15152    0.000555
15153    0.000716
15154    0.005480

[15155 rows x 1 columns]


<ipython-input-7-d067b47df712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undersampleddata=undersampling(readyforundersampling)


         prediction
0      5.779726e-05
1      5.943121e-05
2      1.199340e-05
3      3.631970e-05
4      1.238423e-05
...             ...
21427  4.832173e-07
21428  1.909828e-06
21429  2.311245e-05
21430  1.278244e-04
21431  8.456275e-05

[21432 rows x 1 columns]


<ipython-input-7-d067b47df712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undersampleddata=undersampling(readyforundersampling)


       prediction
0        0.000089
1        0.000891
2        0.000003
3        0.000001
4        0.000022
...           ...
94939    0.018465
94940    0.000456
94941    0.003487
94942    0.002616
94943    0.086798

[94944 rows x 1 columns]


...

<ipython-input-7-d067b47df712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undersampleddata=undersampling(readyforundersampling)


       prediction
0        0.193333
1        0.093249
2        0.656443
3        0.085855
4        0.002835
...           ...
54985    0.000009
54986    0.000031
54987    0.000011
54988    0.000024
54989    0.000023

[54990 rows x 1 columns]


<ipython-input-7-d067b47df712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undersampleddata=undersampling(readyforundersampling)


       prediction
0        0.000506
1        0.001586
2        0.000086
3        0.000075
4        0.000011
...           ...
45787    0.000003
45788    0.000008
45789    0.000001
45790    0.000003
45791    0.000003

[45792 rows x 1 columns]


<ipython-input-7-d067b47df712>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undersampleddata=undersampling(readyforundersampling)


In [51]:
print(undersampleddata)

[            res name|_x  total ASA|_x  b-bone ASA|_x  s-chain ASA|_x  \
468   18014398509481984       159.080        27.9430         131.140   
155       4398046511104        53.782         0.0000          53.782   
1091               4096         0.000         0.0000           0.000   
1000                512        97.980        33.7310          64.249   
793               32768        13.147        13.1470           0.000   
...                 ...           ...            ...             ...   
1173     35184372088832        28.131        28.1310           0.000   
611           134217728        85.728         3.0314          82.697   
145      35184372088832        66.748        66.7480           0.000   
571       4398046511104        76.794         9.2453          67.549   
516     281474976710656         0.340         0.0000           0.340   

      polar ASA|_x  n-polar ASA|_x  total RASA|_x  b-bone RASA|_x  \
468         27.047        132.0400       89.43800         85.5040

In [11]:
os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\undersampled data_no-encoding')
for i in range(len(undersampleddata)):
    os.chdir(r'C:\Users\ahmed hatem\Downloads\Ahmed_Hatem20178003-mohamed khaled-20178056-mohamed-ali-20178036\undersampled data_no-encoding')

    undersampleddata[i].to_csv(labelname[i])

In [8]:
print(len(undersampleddata))

81


In [17]:
pprint(undersampleddata)

Pretty printing has been turned OFF


In [18]:
print(undersampleddata)

[     res name|_x  total ASA|_x  b-bone ASA|_x  s-chain ASA|_x  polar ASA|_x  \
741            C        4.4881         0.3497          4.1384        0.0000   
967            Y      117.3800        29.6910         87.6920       40.0370   
997            A       66.6380        31.6990         34.9390       14.9710   
682            T      124.7600        37.9930         86.7670       50.5820   
325            D       39.6500         3.8375         35.8130       22.5290   
...          ...           ...            ...             ...           ...   
501            N       25.3190         3.9582         21.3600        6.6158   
905            T       52.8850        11.2610         41.6240        6.1703   
690            G       49.1260        49.1260          0.0000       21.1270   
1079           C       15.6520         6.0322          9.6194        1.0330   
786            C       13.1470        13.1470          0.0000       10.7220   

      n-polar ASA|_x  total RASA|_x  b-bone RASA|_